In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%load_ext google.cloud.bigquery
from google.cloud import bigquery

client=bigquery.Client()
project_id = "som-nero-phi-jonc101"

In [ ]:

table_id_base_cohort_with_labels_exlucsion = f"{project_id}.blood_culture_stewardship_sandy_2024.enriched_label_filtered"

In [ ]:
query = f"""
WITH demos AS (
    SELECT 
        c.*,
        demo.gender,

        -- BMI capping logic (replace 10 and 80 with your preferred thresholds)
        CASE 
            WHEN demo.bmi IS NULL THEN NULL
            WHEN demo.bmi < 0 THEN NULL          -- too low → set null
            WHEN demo.bmi > 200 THEN NULL          -- too high → set null
            ELSE demo.bmi                         -- keep valid BMI
        END AS bmi,

        CASE 
            WHEN demo.birth_date_jittered_utc IS NOT NULL 
            THEN DATE_DIFF(DATE(c.blood_culture_order_datetime_utc), DATE(demo.birth_date_jittered_utc), DAY) / 365.0
            ELSE NULL
        END AS age,

        CASE
            WHEN demo.birth_date_jittered_utc IS NOT NULL
            THEN DATE_DIFF(DATE(c.blood_culture_order_datetime_utc), DATE(demo.birth_date_jittered_utc), DAY)
            ELSE NULL
        END AS age_days

    FROM `{table_id_base_cohort_with_labels_exlucsion}` c
    LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.demographic` demo
    ON c.anon_id = demo.anon_id
)

SELECT *
FROM demos
WHERE age <= 18
"""

PEDs_ED_BCx_order_cohort_strict_bmi_age_gender= client.query(query).to_dataframe()

In [ ]:
PEDs_ED_BCx_order_cohort_strict_bmi_age_gender

In [ ]:
# print("this is enriched label for adult only and prevalence at set level")
# PEDs_ED_BCx_order_cohort_strict_bmi_age_gender["set_level_label"].value_counts(normalize=True)

# print("this is enriched label for adult only and prevalence at encounter level")

# PEDs_ED_BCx_order_cohort_strict_bmi_age_gender.drop_duplicates(subset=["anon_id", "pat_enc_csn_id_coded"])["final_label"].value_counts(normalize=True)

In [ ]:
df_sorted = PEDs_ED_BCx_order_cohort_strict_bmi_age_gender.sort_values(
    ['anon_id', 'pat_enc_csn_id_coded', 'blood_culture_order_datetime_utc']
)

PEDs_ED_BCx_order_cohort_strict_bmi_age_gender_latest = (
    df_sorted
    .groupby(['anon_id', 'pat_enc_csn_id_coded'], as_index=False)
    .tail(1)
    .reset_index(drop=True)
)


In [ ]:

# table_id_analyse_cohort_withfinal_label_peds = f"{project_id}.blood_culture_stewardship_sandy_2024.enriched_label_filtered_peds_only_analysis_cohort"

# # Upload the DataFrame to BigQuery
# PEDs_ED_BCx_order_cohort_strict_bmi_age_gender_latest.to_gbq(
#     destination_table=table_id_analyse_cohort_withfinal_label_peds,
#     project_id=project_id,
#     if_exists='replace'  # This will replace the table if it exists
# )

# print(f"Uploaded PEDs_ED_BCx_order_cohort_strict_bmi_age_gender to {table_id_analyse_cohort_withfinal_label_peds}")

In [ ]:
test_cohort = PEDs_ED_BCx_order_cohort_strict_bmi_age_gender_latest[PEDs_ED_BCx_order_cohort_strict_bmi_age_gender_latest["order_year"] >= 2023]

In [ ]:
test_cohort = PEDs_ED_BCx_order_cohort_strict_bmi_age_gender_latest[PEDs_ED_BCx_order_cohort_strict_bmi_age_gender_latest["order_year"] >= 2021]

In [ ]:
test_cohort["final_label"].value_counts()

In [ ]:
# df has a column 'label' where 1=positive, 0=negative
positive_sample = test_cohort[test_cohort['final_label'] == "positive"].sample(n=60, random_state=42)
negative_sample = test_cohort[test_cohort['final_label'] != "positive"].sample(n=40, random_state=42)

balanced_300 = pd.concat([positive_sample, negative_sample]).sample(frac=1, random_state=42)

In [ ]:
query = """
WITH unique_mrn AS (
    SELECT anon_id
    FROM `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-07-17`
    GROUP BY anon_id
    HAVING COUNT(*) = 1
)
SELECT DISTINCT mrn.*
FROM `som-nero-phi-jonc101-secure.starr_map.shc_map_2025-07-17` AS mrn
JOIN unique_mrn u ON mrn.anon_id = u.anon_id
JOIN `som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.enriched_label_filtered_peds_only_analysis_cohort` AS cohort
  ON mrn.anon_id = cohort.anon_id
WHERE cohort.order_year >= 2021
"""
mrn_mapping_test_cohort = client.query(query).to_dataframe()

In [ ]:
mrn_mapping_test_cohort

In [ ]:
test_cohort_with_mrn = test_cohort.merge(mrn_mapping_test_cohort, on="anon_id", how="inner")

In [ ]:
# if jitter might be a string column, coerce to numeric first
test_cohort_with_mrn["jitter_days"] = pd.to_numeric(test_cohort_with_mrn["jitter"], errors="coerce")

test_cohort_with_mrn["candidate_epic_time"] = (
    test_cohort_with_mrn["blood_culture_order_datetime"]
    - pd.to_timedelta(test_cohort_with_mrn["jitter_days"], unit="D")  # negative values fine
    # - pd.Timedelta(hours=8)
)

In [ ]:
columns_to_keep = ["anon_id", "mrn", "pat_enc_csn_id_coded","blood_culture_order_datetime", "blood_culture_order_datetime_utc", "jitter_days","candidate_epic_time", "final_label"]

In [ ]:
test_cohort_with_mrn_columns_to_keep = test_cohort_with_mrn[columns_to_keep]

In [ ]:
positive_sample = test_cohort_with_mrn_columns_to_keep[test_cohort_with_mrn_columns_to_keep['final_label'] == "positive"].sample(n=180, random_state=42)
negative_sample = test_cohort_with_mrn_columns_to_keep[test_cohort_with_mrn_columns_to_keep['final_label'] != "positive"].sample(n=120, random_state=42)

balanced_300 = pd.concat([positive_sample, negative_sample]).sample(frac=1, random_state=42)

In [ ]:
balanced_300

In [ ]:
balanced_300.to_csv("../data/peds_mrn_mapping_balanced_300.csv", index=False)

In [ ]:
# import pandas as pd
# import numpy as np
# pd.set_option('display.max_columns', None)
# pd.read_csv("../data/balanced_100.csv")

